# Tarea 03 - Análisis de datos geoespaciales mediante pandas, plotly, geopandas y folium

## Estudiantes:

Daniel Salazar Mora - B87214
Stephanie María Leitón Ramírez - B74106


In [ ]:
import os
import fiona
import fiona.crs
import folium
import geopandas as gpad
import plotly.express as px

## Obtención de datos

Se usan las capas de Web Feature Service (WFS) publicadas por el Instituto Geográfico Nacional (IGN) en el Sistema Nacional de Información Territorial (SNIT):

Límite cantonal 1:5000
https://www.snitcr.go.cr/ico_servicios_ogc_info?k=bm9kbzo6MjY=&nombre=IGN%20Cartograf%C3%ADa%201:5mil
 
Red vial 1:200000
https://www.snitcr.go.cr/ico_servicios_ogc_info?k=bm9kbzo6NDI=&nombre=IGN%201:200mil


## Tabla con los 82 cantones de Costa Rica

In [ ]:
#cargar como geopandas
redvial_file = gpad.read_file('../datos/redvial.geojson')
cantones_file = gpad.read_file('../datos/cantones.geojson')
#convertir capas
redvial_file.to_crs(5367)
cantones_file.to_crs(5367)

#Unir las capas con intersección espacial 
redvial_x_canton = cantones_file.overlay(redvial_file, how='intersection', keep_geom_type=False)
redvial_x_canton['longitud'] = redvial_x_canton['geometry'].length /1000

In [ ]:
# Group by de canton y categorias
redvial_x_canton_agrupados = redvial_x_canton.groupby(['canton', 'categoria']).agg({'longitud': 'sum', 'area': 'min'})
redvial_x_canton_agrupados = redvial_x_canton_agrupados.reset_index()

# Se separan las categorias como columnas al añadir un multinivel
redvial_x_canton_agrupados = redvial_x_canton_agrupados.pivot_table('longitud', ['canton', 'area'], 'categoria')

# reordenar columnas volviendo a un sólo nivel de columnas
redvial_x_canton_agrupados.reset_index( drop=False, inplace=True )
redvial_x_canton_agrupados.reindex(['CAMINO DE TIERRA','AUTOPISTA',
                                        'CARRETERA PAVIMENTO DOS VIAS O MAS',
                                        'CARRETERA PAVIMENTO UNA VIA',
                                        'CARRETERA SIN PAVIMENTO DOS VIAS'], axis=1)

# calcular longitud total
redvial_x_canton_agrupados['longitud'] = redvial_x_canton_agrupados[['CAMINO DE TIERRA','AUTOPISTA',
                                                                        'CARRETERA PAVIMENTO DOS VIAS O MAS',
                                                                        'CARRETERA PAVIMENTO UNA VIA',
                                                                        'CARRETERA SIN PAVIMENTO DOS VIAS'
                                                                       ]].sum(axis=1)


#calcular densidad
redvial_x_canton_agrupados['densidad'] = redvial_x_canton_agrupados["longitud"].div(redvial_x_canton_agrupados['area'])

redvial_x_canton_agrupados

## Gráfico plotly de barras 

In [ ]:
#Extraer las 15 cantones de mayor longitud total de red vial
# Dataframe filtrado para usar en graficación
longitud_grafico = redvial_x_canton_agrupados[['canton', 'longitud','CAMINO DE TIERRA','AUTOPISTA',
                                                'CARRETERA PAVIMENTO DOS VIAS O MAS',
                                                'CARRETERA PAVIMENTO UNA VIA',
                                                'CARRETERA SIN PAVIMENTO DOS VIAS'
                                              ]].sort_values("longitud", ascending=[False]).head(15)
#Gráfico plotly de barras apiladas 
fig = px.bar(longitud_grafico, x="canton", y=['CAMINO DE TIERRA','AUTOPISTA',
                                                'CARRETERA PAVIMENTO DOS VIAS O MAS',
                                                'CARRETERA PAVIMENTO UNA VIA',
                                                'CARRETERA SIN PAVIMENTO DOS VIAS'
                                             ], title="Cantones de mayor longitud total de red vial")
fig.show()

## Gráfico plotly de pastel 

Porcentaje de red vial de los 15 cantones de mayor longitud total de la red vial, con respecto a la longitud de la red vial en todo el país.
La porción (slice) adicional en el gráfico de pastel llamada “Otros cantones”, correspondiente al porcentaje de la suma de la red vial en los 67 cantones restantes."

In [ ]:
#ordenar los valores por longitud
pastel = redvial_x_canton_agrupados[['canton', 'longitud']].sort_values("longitud", ascending=[False])
#reset al index
pastel = pastel.reset_index()
#extraer la longitud de valor 15
longitud_15= pastel['longitud'].loc[14]
#cambiar el canton a los siguientes valores luego del 15
pastel.loc[pastel['longitud'] < longitud_15 , 'canton'] = 'Otros cantones' 
#Gráfico plotly de pastel 
fig = px.pie(pastel, values='longitud', names='canton', title='Porcentaje de los 15 cantones de mayor longitud total de la red vial en el país')
fig.show()

## Mapa folium 

Un mapa folium con las siguientes capas:
* Capa base (OpenStreetMap, Stamen, etc.).
* Capa de coropletas correspondiente a la densidad de la red vial en los cantones.
* Líneas de la red vial.

Y los siguientes controles:
* Control para activar y desactivar capas.
* Escala


In [ ]:
# Cargar el GeoJason en un dataframe de pandas
densidad_file = redvial_x_canton_agrupados
# Cargar el GeoJason en un dataframe de pandas
cantones_file = gpad.read_file('../datos/cantones.geojson')
# Cargar el GeoJason en un dataframe de pandas
redvial_file = gpad.read_file('../datos/redvial.geojson')

In [ ]:
cantones_file = cantones_file.to_crs(4326)
redvial_file = redvial_file.to_crs(4326)

# Creación del mapa base
mapa= folium.Map(
    location=[9.8, -84], 
    width=1000, height=1000, 
    zoom_start=8,
    control_scale=True,
    tiles='Stamen Watercolor'
    )

#Añadir mapa de coropletas
folium.Choropleth(
    name="Densidad de la red vial en los cantones de Costa Rica",
    geo_data=cantones_file,
    data=densidad_file,
    columns=['canton', 'densidad'],
    bins=8,
    key_on='feature.properties.canton',
    fill_color='Greens', 
    fill_opacity=1, 
    line_opacity=1,
    legend_name='densidad',
    smooth_factor=0 ).add_to(mapa)

#añadir capa con las lineas de red vial
folium.GeoJson(data=redvial_file, name='Red vial').add_to(mapa)

# Control de capas
folium.LayerControl().add_to(mapa)

# Despliegue del mapa
mapa